In [8]:
import pandas as pd  
import kagglehub

In [9]:
path = kagglehub.dataset_download("himelsarder/road-accident-survival-dataset")
print("Path to dataset files:", path)
file_path = f"{path}/accident.csv" 

Path to dataset files: C:\Users\gsocc\.cache\kagglehub\datasets\himelsarder\road-accident-survival-dataset\versions\1


In [10]:
df = pd.read_csv(file_path)

df_cleaned = df.dropna(how='all')
df_cleaned.to_csv(f"{path}/cleaned_dataset.csv", index=False)
print(df_cleaned.head())

   Age  Gender  Speed_of_Impact Helmet_Used Seatbelt_Used  Survived
0   56  Female             27.0          No            No         1
1   69  Female             46.0          No           Yes         1
2   46    Male             46.0         Yes           Yes         0
3   32    Male            117.0          No           Yes         0
4   60  Female             40.0         Yes           Yes         0
